<h1> MODEL IMPLEMENTATIONS <h1>

<h2> Imports <h2>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import cupy as cp
import sklearn

<h2> Import and Split <h2>

In [3]:
path = r"Dataset\articles.csv"
data = pd.read_csv(path)
# print(data.head())

In [4]:
X = data['content']
Y = data['gold_label']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")


category_to_label = {
    "entertainment": 0,
    "business": 1,
    "sports": 2,
    "science-technology": 3,
    "international": 4
}

Y_train = [category_to_label[label.strip()] for label in Y_train if label.strip() in category_to_label]
Y_test = [category_to_label[label.strip()] for label in Y_test if label.strip() in category_to_label]


X_train shape: (1594,)
X_test shape: (399,)
Y_train shape: (1594,)
Y_test shape: (399,)


<h2> Bag of Words <h2>


In [64]:
class TextProcessor:
    def __init__(self, dataset):
        self.dataset = dataset
        self.vocabulary = {}
    
    def build_vocabulary(self):
        unique_words = set()
        for sentence in self.dataset:
            unique_words.update(sentence.split())
        self.vocabulary = {word: idx for idx, word in enumerate(sorted(unique_words))}
        return self.vocabulary
    
    def sentence_to_bow(self, sentence):
        words = sentence.split()
        vector = np.zeros(len(self.vocabulary))
        for word in words:
            if word in self.vocabulary:
                index = self.vocabulary[word]
                # print(index)
                vector[index] += 1
        return vector
    
    def vectorize_sentences(self, X):
        return [self.sentence_to_bow(sentence) for sentence in X]


In [65]:
bag = TextProcessor(X_train)
vocab = bag.build_vocabulary()
train_x = bag.vectorize_sentences(X_train)
test_x = bag.vectorize_sentences(X_test)
# print("trainx", train_x[0][12242])
# print(vocab)

<h2> Neural Network Using Pytorch <h2>

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
train_x = torch.tensor(train_x, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.long)
Y_test = torch.tensor(Y_test, dtype=torch.long)

batch_size = 64
train_dataset = TensorDataset(train_x, Y_train)
test_dataset = TensorDataset(test_x, Y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [43]:
import torch.nn as nn
import torch.nn.functional as F
class NewsClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NewsClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.dropout = nn.Dropout(0.5) 
        self.fc4 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  
        x = self.fc4(x)
        return x


In [60]:
input_size = train_x.shape[1]
num_classes = len(Y_train.unique())
model = NewsClassifier(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)


In [61]:
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_x, batch_y in train_loader:
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/15], Loss: 1.5415
Epoch [2/15], Loss: 1.3203
Epoch [3/15], Loss: 1.0694
Epoch [4/15], Loss: 0.8575
Epoch [5/15], Loss: 0.6859
Epoch [6/15], Loss: 0.5591
Epoch [7/15], Loss: 0.4706
Epoch [8/15], Loss: 0.3943
Epoch [9/15], Loss: 0.3349
Epoch [10/15], Loss: 0.2903
Epoch [11/15], Loss: 0.2567
Epoch [12/15], Loss: 0.2264
Epoch [13/15], Loss: 0.2011
Epoch [14/15], Loss: 0.1759
Epoch [15/15], Loss: 0.1601


In [62]:
def evaluate_model(loader, model):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch_x, batch_y in loader:
            outputs = model(batch_x)
            _, predicted = torch.max(outputs, 1)  
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    
    accuracy = correct / total
    return accuracy

train_accuracy = evaluate_model(train_loader, model)
test_accuracy = evaluate_model(test_loader, model)
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")


Training Accuracy: 0.99
Test Accuracy: 0.97


Use these for next models, all are in vector forms already and labels are also in represented numerically in Y_test, Y_train.

In [ ]:
train_x,test_x,Y_test,Y_train
vocab # for vocabulary of words